# Basic Map

In [6]:
# --- Imports & setup
import os, glob, warnings
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from shapely.ops import unary_union
from fiona.env import Env
import pandas as pd

# --- Paths (updated)
DATA_DIR   = r"C:\Users\HP\Documents\SpatialCARE\Daily\DailyGPKG"
PASIG_SHP  = r"C:\Users\HP\Documents\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
ROADS_SHP  = r"C:\Users\HP\Documents\PhD Class\Shapefile\MM\Pasig\PasigRN.shp"
OUT_DIR    = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures"
os.makedirs(OUT_DIR, exist_ok=True)

# --- CRS & figure
WGS84  = "+proj=longlat +datum=WGS84 +no_defs"
UTM51  = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7,7), 150

def dedupe_points(gdf):
    t = gdf.to_crs(UTM51).copy()
    t["x"] = t.geometry.x; t["y"] = t.geometry.y
    return t.drop_duplicates(subset=["x","y"]).to_crs(UTM51)

# --- Read Pasig boundary
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None: 
    pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)

# --- Read and filter Pasig road network
with Env(SHAPE_RESTORE_SHX="YES"):
    roads = gpd.read_file(ROADS_SHP)
if roads.crs is None: 
    roads = roads.set_crs(WGS84)

# choose attribute column for road class
col = None
for candidate in ("highway", "fclass", "type"):
    if candidate in roads.columns:
        col = candidate
        break
if col is None:
    raise SystemExit("No 'highway'/'fclass'/'type' column found in road shapefile.")

wanted = {"primary", "secondary", "tertiary"}
roads[col] = roads[col].astype(str).str.lower()
roads_sel = roads[roads[col].isin(wanted)].copy()

# project and clip to Pasig boundary
roads_sel_utm = roads_sel.to_crs(UTM51)
try:
    roads_clip = gpd.clip(roads_sel_utm, pasig_utm)
except Exception:
    roads_clip = gpd.overlay(roads_sel_utm, pasig_utm[["geometry"]], how="intersection")

# --- Collect stations from all daily files
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files:
    raise SystemExit("No daily GPKG files found.")

pts = []
for f in files:
    g = gpd.read_file(f)
    if g.crs is None: 
        g = g.set_crs(WGS84)
    if "geometry" not in g or g.geometry.is_empty.all():
        lon = [c for c in g.columns if str(c).lower() in ("longitude","lon","x")]
        lat = [c for c in g.columns if str(c).lower() in ("latitude","lat","y")]
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon[0]], g[lat[0]], crs=g.crs))
    pts.append(g)

all_pts = gpd.GeoDataFrame(pd.concat(pts, ignore_index=True), geometry="geometry", crs=pts[0].crs)
stations = dedupe_points(all_pts)  # returns UTM51

# --- Plot
fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)

# Color palette
palette = {
    "primary":   "#d62728",  # red
    "secondary": "#1f77b4",  # blue
    "tertiary":  "#2ca02c",  # green
    "sensors":   "#9467bd"   # purple
}

# Pasig boundary
pasig_utm.boundary.plot(ax=ax, linewidth=0.8, color="#444444", zorder=1)

# Roads: draw by class with distinct colors
style_map = {
    "primary":   {"linewidth": 1.8, "color": palette["primary"],   "zorder": 2},
    "secondary": {"linewidth": 1.4, "color": palette["secondary"], "zorder": 2},
    "tertiary":  {"linewidth": 1.0, "color": palette["tertiary"],  "zorder": 2},
}
for klass, style in style_map.items():
    subset = roads_clip.loc[roads_clip[col] == klass]
    if not subset.empty:
        subset.plot(ax=ax, label=klass.title(), **style)

# Sensors: distinct symbol
stations.plot(
    ax=ax,
    color=palette["sensors"],
    markersize=50,
    marker="o",
    edgecolor="white",
    linewidth=0.8,
    zorder=3,
    label="Sensors"
)

# Title and Legend outside the map
ax.set_title("Pasig — Primary/Secondary/Tertiary Roads & Station Locations")
ax.legend(
    title="Legend",
    frameon=False,
    loc="upper left",
    bbox_to_anchor=(1.02, 1),
    borderaxespad=0
)
ax.set_axis_off()

out = os.path.join(OUT_DIR, "pasig_roads_primary_secondary_tertiary_with_stations.png")
plt.tight_layout()
plt.savefig(out, bbox_inches="tight")
plt.close(fig)
print("Saved:", out)

# --- Optional: also write filtered roads to file for reuse
roads_out = os.path.join(OUT_DIR, "Pasig_Roads_PriSecTer.gpkg")
roads_clip.to_crs(WGS84).to_file(roads_out, driver="GPKG")
print("Filtered roads saved to:", roads_out)


Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\pasig_roads_primary_secondary_tertiary_with_stations.png
Filtered roads saved to: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\Pasig_Roads_PriSecTer.gpkg


# Map with Basemap and Roads

In [4]:
# --- Imports & setup
import os, glob, warnings
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from shapely.ops import unary_union
from fiona.env import Env
import fiona  # for safe layer removal in GPKG
import pandas as pd
import contextily as ctx  # for basemap tiles
from datetime import datetime

# --- Paths (updated)
DATA_DIR   = r"C:\Users\HP\Documents\SpatialCARE\Daily\DailyGPKG"
PASIG_SHP  = r"C:\Users\HP\Documents\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
ROADS_SHP  = r"C:\Users\HP\Documents\PhD Class\Shapefile\MM\Pasig\PasigRN.shp"
OUT_DIR    = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures"
os.makedirs(OUT_DIR, exist_ok=True)

# --- CRS & figure
WGS84   = "+proj=longlat +datum=WGS84 +no_defs"
UTM51   = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
WEB_MCT = "EPSG:3857"  # Web Mercator for basemaps
FIG_SIZE, FIG_DPI = (7,7), 150

def dedupe_points(gdf):
    # de-duplicate by projected coords; returns UTM51
    t = gdf.to_crs(UTM51).copy()
    t["x"] = t.geometry.x; t["y"] = t.geometry.y
    return t.drop_duplicates(subset=["x","y"]).to_crs(UTM51)

def safe_write_gpkg(gdf, out_path, layer_name="layer"):
    """Write GeoDataFrame to GPKG, replacing the layer if it exists.
       Falls back to timestamped file if overwrite fails."""
    gdf = gdf.to_crs(WGS84)

    if os.path.exists(out_path):
        try:
            layers = fiona.listlayers(out_path)
            if layer_name in layers:
                fiona.remove(out_path, layer=layer_name, driver="GPKG")
        except Exception:
            base, ext = os.path.splitext(out_path)
            backup = f"{base}_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}{ext}"
            os.replace(out_path, backup)
            print(f"[info] Backed up existing GPKG to: {backup}")

    try:
        gdf.to_file(out_path, driver="GPKG", layer=layer_name)
        print(f"Filtered roads saved to: {out_path} (layer='{layer_name}')")
    except Exception as e:
        alt = out_path.replace(".gpkg", f"_{datetime.now().strftime('%Y%m%d_%H%M%S')}.gpkg")
        print(f"[warn] Primary write failed due to: {e}\n       Writing to new file: {alt}")
        gdf.to_file(alt, driver="GPKG", layer=layer_name)
        print(f"Filtered roads saved to: {alt} (layer='{layer_name}')")

# --- Read Pasig boundary
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None:
    pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)

# --- Read and filter Pasig road network
with Env(SHAPE_RESTORE_SHX="YES"):
    roads = gpd.read_file(ROADS_SHP)
if roads.crs is None:
    roads = roads.set_crs(WGS84)

# choose attribute column for road class
col = None
for candidate in ("highway", "fclass", "type"):
    if candidate in roads.columns:
        col = candidate
        break
if col is None:
    raise SystemExit("No 'highway'/'fclass'/'type' column found in road shapefile.")

wanted = {"primary", "secondary", "tertiary"}
roads[col] = roads[col].astype(str).str.lower()
roads_sel = roads[roads[col].isin(wanted)].copy()

# project and clip to Pasig boundary (UTM for robust geometry ops)
roads_sel_utm = roads_sel.to_crs(UTM51)
try:
    roads_clip = gpd.clip(roads_sel_utm, pasig_utm)
except Exception:
    roads_clip = gpd.overlay(roads_sel_utm, pasig_utm[["geometry"]], how="intersection")

# --- Collect stations from all daily files
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files:
    raise SystemExit("No daily GPKG files found.")

pts = []
for f in files:
    g = gpd.read_file(f)
    if g.crs is None:
        g = g.set_crs(WGS84)
    if "geometry" not in g or g.geometry.is_empty.all():
        lon = [c for c in g.columns if str(c).lower() in ("longitude","lon","x")]
        lat = [c for c in g.columns if str(c).lower() in ("latitude","lat","y")]
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon[0]], g[lat[0]], crs=g.crs))
    pts.append(g)

all_pts = gpd.GeoDataFrame(pd.concat(pts, ignore_index=True), geometry="geometry", crs=pts[0].crs)
stations = dedupe_points(all_pts)  # UTM51

# --- Reproject display layers to Web Mercator for the basemap
pasig_web     = pasig_utm.to_crs(WEB_MCT)
roads_web     = roads_clip.to_crs(WEB_MCT)
stations_web  = stations.to_crs(WEB_MCT)

# Quick debug: counts (optional)
print("Counts — primary/secondary/tertiary:",
      (roads_web[col] == "primary").sum(),
      (roads_web[col] == "secondary").sum(),
      (roads_web[col] == "tertiary").sum())

# --- Plot
fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)

# 1) Plot vector data FIRST (so extent is set by data)
# Color palette
palette = {
    "primary":   "#d62728",  # red
    "secondary": "#1f77b4",  # blue
    "tertiary":  "#2ca02c",  # green
    "sensors":   "#9467bd"   # purple
}

# Pasig boundary (thin line so imagery remains visible)
pasig_web.boundary.plot(ax=ax, linewidth=0.9, color="#222222", zorder=2)

# Roads by class
style_map = {
    "primary":   {"linewidth": 2.0, "color": palette["primary"],   "zorder": 3},
    "secondary": {"linewidth": 1.6, "color": palette["secondary"], "zorder": 3},
    "tertiary":  {"linewidth": 1.2, "color": palette["tertiary"],  "zorder": 3},
}
for klass, style in style_map.items():
    subset = roads_web.loc[roads_web[col] == klass]
    if not subset.empty:
        subset.plot(ax=ax, label=klass.title(), **style)

# Sensors
stations_web.plot(
    ax=ax,
    color=palette["sensors"],
    markersize=55,
    marker="o",
    edgecolor="white",
    linewidth=0.9,
    zorder=4,
    label="Sensors"
)

# 2) Fix extent using Pasig bounds (with a small padding)
minx, miny, maxx, maxy = pasig_web.total_bounds
pad_x = (maxx - minx) * 0.03
pad_y = (maxy - miny) * 0.03
ax.set_xlim(minx - pad_x, maxx + pad_x)
ax.set_ylim(miny - pad_y, maxy + pad_y)

# 3) Add basemap BEHIND the vectors; do NOT reset extent
ctx.add_basemap(
    ax,
    source=ctx.providers.Esri.WorldImagery,
    crs=pasig_web.crs,
    zoom=14,
    reset_extent=False,
    zorder=0
)

# Title and Legend outside the map
ax.set_title("Pasig — Primary/Secondary/Tertiary Roads & Station Locations")
ax.legend(
    title="Legend",
    frameon=False,
    loc="upper left",
    bbox_to_anchor=(1.02, 1),
    borderaxespad=0
)
ax.set_axis_off()

out = os.path.join(OUT_DIR, "pasig_roads_primary_secondary_tertiary_with_stations_basemap.png")
plt.tight_layout()
plt.savefig(out, bbox_inches="tight")
plt.close(fig)
print("Saved figure:", out)

# --- Save filtered roads safely to GPKG (WGS84)
roads_out   = os.path.join(OUT_DIR, "Pasig_Roads_PriSecTer.gpkg")
layer_name  = "Pasig_Roads_PriSecTer"
safe_write_gpkg(roads_clip, roads_out, layer_name=layer_name)

SystemExit: No daily GPKG files found.

C:\Users\HP\anaconda3\envs\CARESpatial\Lib\site-packages\IPython\core\interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
# --- Imports & setup
import os, glob, warnings, re
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from shapely.ops import unary_union
from fiona.env import Env
import fiona  # for safe layer removal in GPKG
import pandas as pd
import contextily as ctx  # for basemap tiles
from datetime import datetime
from collections import Counter
import matplotlib.patheffects as pe

# --- Paths
DATA_DIR    = r"C:\Users\HP\Desktop\SpatialCARE\Daily\DailyGPKG"
PASIG_SHP   = r"C:\Users\HP\Desktop\SpatialCARE\Pasig\Pasig.shp"
ROADS_SHP   = r"C:\Users\HP\Desktop\SpatialCARE\Pasig\PasigRN.shp"
OUT_F_DIR   = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures"
os.makedirs(OUT_DIR, exist_ok=True)

# --- CRS & figure
WGS84   = "+proj=longlat +datum=WGS84 +no_defs"
UTM51   = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
WEB_MCT = "EPSG:3857"  # Web Mercator
FIG_SIZE, FIG_DPI = (7,7), 150

# --- Helpers
def _norm(s: str) -> str:
    return re.sub(r'[^a-z0-9]', '', str(s).lower())

def pick_name_column(df: pd.DataFrame):
    """Pick a column for station names (case/space/underscore insensitive)."""
    if df is None or df.empty:
        return None
    norm_map = {_norm(c): c for c in df.columns}
    candidates = [
        "location_name","location",          # added for your case
        "stationname","station","sitename","site",
        "sensorname","sensor","label",
        "stationid","siteid","sensorid","id"
    ]
    for key in candidates:
        if key in norm_map:
            return norm_map[key]
    return None

def dedupe_points_with_name_mode(gdf, name_col=None, xy_round_m=0.5):
    """De-duplicate by rounded coords, keep the most frequent name per location."""
    t = gdf.to_crs(UTM51).copy()
    t["x"] = t.geometry.x
    t["y"] = t.geometry.y
    t["x_r"] = (t["x"]/xy_round_m).round().astype(int)
    t["y_r"] = (t["y"]/xy_round_m).round().astype(int)

    if name_col and name_col in gdf.columns:
        t["_name"] = (
            gdf[name_col]
            .astype(str).str.strip()
            .replace({"nan": np.nan, "None": np.nan, "": np.nan})
        )
    else:
        t["_name"] = np.nan

    records = []
    for (xr, yr), grp in t.groupby(["x_r","y_r"]):
        x, y = grp["x"].iloc[0], grp["y"].iloc[0]
        names = [n for n in grp["_name"].tolist() if pd.notna(n)]
        if names:
            mode_name = Counter(names).most_common(1)[0][0]
            records.append((x,y,mode_name))
        else:
            records.append((x,y,None))

    out = gpd.GeoDataFrame(
        {"station_name": [n if n else None for (_,_,n) in records]},
        geometry=gpd.points_from_xy([r[0] for r in records],
                                    [r[1] for r in records],
                                    crs=UTM51)
    )
    # fill missing with fallback IDs
    missing = out["station_name"].isna()
    if missing.any():
        idxs = np.flatnonzero(missing)
        for i, ridx in enumerate(idxs, start=1):
            out.at[ridx,"station_name"] = f"Station_{i:03d}"
    return out

def safe_write_gpkg(gdf, out_path, layer_name="layer"):
    gdf = gdf.to_crs(WGS84)
    if os.path.exists(out_path):
        try:
            layers = fiona.listlayers(out_path)
            if layer_name in layers:
                fiona.remove(out_path, layer=layer_name, driver="GPKG")
        except Exception:
            base, ext = os.path.splitext(out_path)
            backup = f"{base}_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}{ext}"
            os.replace(out_path, backup)
            print(f"[info] Backed up existing GPKG to: {backup}")

    try:
        gdf.to_file(out_path, driver="GPKG", layer=layer_name)
    except Exception as e:
        alt = out_path.replace(".gpkg", f"_{datetime.now().strftime('%Y%m%d_%H%M%S')}.gpkg")
        print(f"[warn] Primary write failed: {e}\nWriting to {alt}")
        gdf.to_file(alt, driver="GPKG", layer=layer_name)

# --- Read Pasig boundary
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None:
    pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)

# --- Read and filter Pasig road network
with Env(SHAPE_RESTORE_SHX="YES"):
    roads = gpd.read_file(ROADS_SHP)
if roads.crs is None:
    roads = roads.set_crs(WGS84)

col = None
for c in ("highway","fclass","type"):
    if c in roads.columns: col=c; break
if col is None: raise SystemExit("No road class column found.")

wanted = {"primary","secondary","tertiary"}
roads[col] = roads[col].astype(str).str.lower()
roads_sel = roads[roads[col].isin(wanted)].copy()

roads_sel_utm = roads_sel.to_crs(UTM51)
try:
    roads_clip = gpd.clip(roads_sel_utm, pasig_utm)
except Exception:
    roads_clip = gpd.overlay(roads_sel_utm, pasig_utm[["geometry"]], how="intersection")

# --- Collect stations
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files: raise SystemExit("No daily GPKG files found.")

pts=[]
for f in files:
    g=gpd.read_file(f)
    if g.crs is None: g=g.set_crs(WGS84)
    if "geometry" not in g or g.geometry.is_empty.all():
        lon=[c for c in g.columns if _norm(c) in ("longitude","lon","x")]
        lat=[c for c in g.columns if _norm(c) in ("latitude","lat","y")]
        if lon and lat:
            g=g.set_geometry(gpd.points_from_xy(g[lon[0]], g[lat[0]], crs=g.crs))
    pts.append(g)

all_pts=gpd.GeoDataFrame(pd.concat(pts, ignore_index=True),
                         geometry="geometry", crs=pts[0].crs)

# Prefer location_name column
if "location_name" in all_pts.columns:
    name_col = "location_name"
else:
    name_col = pick_name_column(all_pts)
print(f"[info] Using station name column: {name_col}")

stations=dedupe_points_with_name_mode(all_pts, name_col=name_col, xy_round_m=0.5)

# --- Reproject for plotting
pasig_web=pasig_utm.to_crs(WEB_MCT)
roads_web=roads_clip.to_crs(WEB_MCT)
stations_web=stations.to_crs(WEB_MCT)

# --- Plot
fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
palette={"primary":"#d62728","secondary":"#1f77b4","tertiary":"#2ca02c","sensors":"#9467bd"}

pasig_web.boundary.plot(ax=ax, linewidth=0.9, color="#222222", zorder=2)

style_map={"primary":{"linewidth":2.0,"color":palette["primary"],"zorder":3},
           "secondary":{"linewidth":1.6,"color":palette["secondary"],"zorder":3},
           "tertiary":{"linewidth":1.2,"color":palette["tertiary"],"zorder":3}}

for klass,style in style_map.items():
    subset=roads_web.loc[roads_web[col]==klass]
    if not subset.empty: subset.plot(ax=ax,label=klass.title(),**style)

stations_web.plot(ax=ax,color=palette["sensors"],markersize=55,marker="o",
                  edgecolor="white",linewidth=0.9,zorder=4,label="Sensors")

# Labels
for _, row in stations_web.iterrows():
    x,y=row.geometry.x,row.geometry.y
    name=str(row["station_name"])
    ax.text(x,y," "+name,fontsize=8,va="center",ha="left",zorder=5,
            path_effects=[pe.withStroke(linewidth=2.5, foreground="white", alpha=0.9)])

# Fix extent
minx,miny,maxx,maxy=pasig_web.total_bounds
pad_x=(maxx-minx)*0.03; pad_y=(maxy-miny)*0.03
ax.set_xlim(minx-pad_x,maxx+pad_x); ax.set_ylim(miny-pad_y,maxy+pad_y)

ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery,
                crs=pasig_web.crs, zoom=14, reset_extent=False, zorder=0)

ax.set_title("Pasig — Primary/Secondary/Tertiary Roads & Station Locations")
ax.legend(title="Legend", frameon=False, loc="upper left",
          bbox_to_anchor=(1.02,1), borderaxespad=0)
ax.set_axis_off()

out=os.path.join(OUT_DIR,"pasig_roads_with_station_names.png")
plt.tight_layout(); plt.savefig(out,bbox_inches="tight"); plt.close(fig)
print("Saved figure:", out)

# --- Save roads
roads_out=os.path.join(OUT_DIR,"Pasig_Roads_PriSecTer.gpkg")
safe_write_gpkg(roads_clip, roads_out, layer_name="Pasig_Roads_PriSecTer")

[info] Using station name column: None
Saved figure: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\pasig_roads_with_station_names.png
